In [1]:
#importing the modules

import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import os
import matplotlib  



In [ ]:
import Augmentor
import os

input_output_folder1 = "D:\\Varun Robo 2\\Covid19-dataset\\train\\Covid"
input_output_folder2 = "D:\\Varun Robo 2\\Covid19-dataset\\train\\Normal"
input_output_folder3 = "D:\\Varun Robo 2\\Covid19-dataset\\train\\Viral Pneumonia"


p1 = Augmentor.Pipeline(source_directory=input_output_folder1, output_directory=input_output_folder1)
p2 = Augmentor.Pipeline(source_directory=input_output_folder2, output_directory=input_output_folder2)
p3 = Augmentor.Pipeline(source_directory=input_output_folder3, output_directory=input_output_folder3)

#defining augmentation operations

augmentation_operations = [
    lambda p: p.rotate(probability=0.9, max_left_rotation=25, max_right_rotation=25),
    lambda p: p.random_distortion(probability=0.8, grid_width=3, grid_height=3, magnitude=16),
    lambda p: p.random_erasing(probability=0.7, rectangle_area=0.4),
    lambda p: p.flip_left_right(probability=0.8),
    lambda p: p.flip_top_bottom(probability=0.8),
    lambda p: p.zoom_random(probability=0.8, percentage_area=0.6),
    lambda p: p.random_contrast(probability=0.7, min_factor=0.5, max_factor=1.5),
    lambda p: p.random_brightness(probability=0.7, min_factor=0.5, max_factor=1.5),
    lambda p: p.skew(probability=0.6, magnitude=0.6),
    lambda p: p.shear(probability=0.6, max_shear_left=20, max_shear_right=20),
    lambda p: p.random_color(probability=0.5, min_factor=0.5, max_factor=2.0)
]



for operation in augmentation_operations:
    operation(p1)
    operation(p2)
    operation(p3)

p1.set_save_format("png")
p2.set_save_format("png")
p3.set_save_format("png")

num_samples = 300

p1.sample(num_samples)
p2.sample(num_samples)
p3.sample(num_samples)


In [9]:

#importing the training dataset


import cv2
import os
import numpy as np

covid_dir = "D:\\Varun Robo 2\\Covid19-dataset\\train\\Covid"
normal_dir = "D:\\Varun Robo 2\\Covid19-dataset\\train\\Normal"
viral_dir = "D:\\Varun Robo 2\\Covid19-dataset\\train\\Viral Pneumonia"

#initialising the lists for images and labels

x_train = []
y_train = []

#function to load the images and assign a label

def load_images(folder, label):

    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path)  
        if img is not None: 
            img = cv2.resize(img, (224, 224)) 
            x_train.append(img) 
            y_train.append(label)


# Loading images from directory and assigning labels, 0 for covid, 1 for normal, 2 for viral pneumonia

load_images(covid_dir, 0)
load_images(normal_dir, 1)
load_images(viral_dir, 2)

# Converting lists to numpy arrays

x_train = np.array(x_train)
y_train = np.array(y_train)

print(f"Total images loaded: {len(x_train)}")
print(f"Shape of x_train: {x_train.shape}")
print(f"Shape of y_train: {y_train.shape}")











Total images loaded: 1110
Shape of x_train: (1110, 224, 224, 3)
Shape of y_train: (1110,)


In [10]:
#importing the testing dataset

import cv2
import os
import numpy as np

covid_dir1 = "D:\\Varun Robo 2\\Covid19-dataset\\test\\Covid"
normal_dir1 = "D:\\Varun Robo 2\\Covid19-dataset\\test\\Normal"
viral_dir1 = "D:\\Varun Robo 2\\Covid19-dataset\\test\\Viral Pneumonia"



x_test = []
y_test = []

# Function to load images and assign labels

def load_images(folder, label):
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path)  
        if img is not None: 
            img = cv2.resize(img, (224, 224)) 
            x_test.append(img)
            y_test.append(label)

# Assigning labels 0 for covid images,1 for normal images, 2 for viral pneumonia images

load_images(covid_dir1, 0)  
load_images(normal_dir1, 1)
load_images(viral_dir1, 2)

# Converting the lists to numpy arrays

x_test = np.array(x_test)
y_test = np.array(y_test)

print(f"Total images loaded: {len(x_test)}")
print(f"Shape of x_test: {x_test.shape}")
print(f"Shape of y_test: {y_test.shape}")


Total images loaded: 66
Shape of x_test: (66, 224, 224, 3)
Shape of y_test: (66,)


In [11]:
#normalising the data

x_train = x_train / 255.0
x_test = x_test / 255.0
print(f"Shape of x_train after normalization: {x_train.shape}")
print(f"Shape of x_test after normalization: {x_test.shape}")

Shape of x_train after normalization: (1110, 224, 224, 3)
Shape of x_test after normalization: (66, 224, 224, 3)


In [12]:
from sklearn.model_selection import train_test_split

# Perform the 80/20 split, val dataset is 20 percent of the train dataset

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)


print(f"Shape of x_train: {x_train.shape}")
print(f"Shape of x_val: {x_val.shape}")
print(f"Shape of y_train: {y_train.shape}")
print(f"Shape of y_val: {y_val.shape}")


Shape of x_train: (888, 224, 224, 3)
Shape of x_val: (222, 224, 224, 3)
Shape of y_train: (888,)
Shape of y_val: (222,)


In [17]:
import tensorflow as tf
from tensorflow.keras import regularizers 

def create_model(lr=0.001): 
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D((2, 2)),
        
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D((2, 2)),
        
        tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D((2, 2)),
        
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
        tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(3, activation='softmax', kernel_regularizer=regularizers.l2(0.001))
    ])

   
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    
   
    model.compile(optimizer=optimizer,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model




In [18]:
model = create_model(learning_rate=0.0001)

# Fit the model
history = model.fit(x_train, y_train, 
                    epochs=10,
                    batch_size=32, 
                    validation_data=(x_val, y_val))


Epoch 1/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 104s 3s/step - accuracy: 0.4549 - loss: 3.0357 - val_accuracy: 0.4009 - val_loss: 2.2858
Epoch 2/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 59s 2s/step - accuracy: 0.6261 - loss: 2.3617 - val_accuracy: 0.2973 - val_loss: 2.5284
Epoch 3/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 58s 2s/step - accuracy: 0.6924 - loss: 1.8085 - val_accuracy: 0.3153 - val_loss: 2.8783
Epoch 4/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 57s 2s/step - accuracy: 0.7701 - loss: 1.5146 - val_accuracy: 0.3649 - val_loss: 3.4264
Epoch 5/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 57s 2s/step - accuracy: 0.8182 - loss: 1.4031 - val_accuracy: 0.4550 - val_loss: 3.3347
Epoch 6/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 57s 2s/step - accuracy: 0.8992 - loss: 1.2170 - val_accuracy: 0.3604 - val_loss: 4.1358
Epoch 7/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 57s 2s/step - accuracy: 0.9232 - loss: 1.1160 - val_accuracy: 0.3604 - val_loss: 4.3090
Epoch 8/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 57s 2s/step - accuracy: 0.9273 - loss: 1.0608 - val_accuracy: 0.3604 - val_loss

In [21]:
import tensorflow as tf
from tensorflow.keras import layers, models

num_classes = 3

def create_mobilenet():
    base_model = tf.keras.applications.MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False 
    
    model1 = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(1024, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])
    
    model1.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model1


model1 = create_mobilenet()


history = model1.fit(x_train, y_train, 
                     epochs=10,
                     batch_size=32, 
                     validation_data=(x_val, y_val))


17225924/17225924 ━━━━━━━━━━━━━━━━━━━━ 10s 1us/step
Epoch 1/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 41s 952ms/step - accuracy: 0.3849 - loss: 2.2821 - val_accuracy: 0.7252 - val_loss: 0.6603
Epoch 2/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 19s 654ms/step - accuracy: 0.7082 - loss: 0.6942 - val_accuracy: 0.8018 - val_loss: 0.5202
Epoch 3/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 18s 634ms/step - accuracy: 0.8007 - loss: 0.4939 - val_accuracy: 0.7973 - val_loss: 0.5069
Epoch 4/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 17s 626ms/step - accuracy: 0.8500 - loss: 0.4001 - val_accuracy: 0.8333 - val_loss: 0.4616
Epoch 5/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 17s 624ms/step - accuracy: 0.8480 - loss: 0.3404 - val_accuracy: 0.8288 - val_loss: 0.5048
Epoch 6/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 17s 624ms/step - accuracy: 0.8842 - loss: 0.2854 - val_accuracy: 0.8243 - val_loss: 0.4642
Epoch 7/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 19s 685ms/step - accuracy: 0.9189 - loss: 0.2295 - val_accuracy: 0.8468 - val_loss: 0.4658
Epoch 8/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 18s 628

In [22]:
import tensorflow as tf
from tensorflow.keras import layers, models

num_classes = 3


def create_resnet50():
    base_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False
    
    model2 = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(1024, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])
    
    model2.compile(optimizer='adam',
                   loss='sparse_categorical_crossentropy',
                   metrics=['accuracy'])
    
    return model2


model2 = create_resnet50()


history = model2.fit(x_train, y_train, 
                     epochs=10,
                     batch_size=32, 
                     validation_data=(x_val, y_val))


test_loss, test_acc = model2.evaluate(x_test, y_test)
print(f'Test accuracy: {test_acc}')


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 51s 1us/step
Epoch 1/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 63s 2s/step - accuracy: 0.3413 - loss: 1.6623 - val_accuracy: 0.3919 - val_loss: 1.1242
Epoch 2/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 51s 2s/step - accuracy: 0.3344 - loss: 1.2194 - val_accuracy: 0.3423 - val_loss: 1.0764
Epoch 3/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 49s 2s/step - accuracy: 0.3872 - loss: 1.1356 - val_accuracy: 0.2973 - val_loss: 1.1384
Epoch 4/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 50s 2s/step - accuracy: 0.3853 - loss: 1.1123 - val_accuracy: 0.4910 - val_loss: 1.0278
Epoch 5/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 50s 2s/step - accuracy: 0.4097 - loss: 1.0660 - val_accuracy: 0.4324 - val_loss: 1.0579
Epoch 6/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 49s 2s/step - accuracy: 0.3857 - loss: 1.0807 - val_accuracy: 0.4910 - val_loss: 1.0287
Epoch 7/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 49s 2s/step - accuracy: 0.4034 - loss: 1.0610 - val_accuracy: 0.4955 - val_loss: 1.0099
Epoch 8/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 50s 2s/step - accuracy: 0.44